In [ ]:
#!pip install pandas transformers torch

Special install of pytorch for apple sillicon

In [ ]:
#!pip3 install --pre torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/nightly/cpu

In [ ]:
# Import necessary libraries
import pandas as pd
from transformers import pipeline
import utils

In [ ]:
# Step 1: Load and Parse the possible classes
# =================================

# Load the CSV file with categories and identifiers
categories = pd.read_csv('hierarchy/FoR_code_hierarchical_4_digit.csv')
# Build the Hierarchical Structure with Identifiers
hierarchy_with_ids = utils.build_hierarchy_with_ids(categories)

In [ ]:
# Step 2: Initialize the Hugging Face Model
# ==========================================

# Load the Hugging Face model for classification
classifier = pipeline("zero-shot-classification",
                      #model="facebook/bart-large-mnli")
                      model="MoritzLaurer/deberta-v3-large-zeroshot-v2.0",device = "mps")


In [ ]:
# Step 3: Load the CSV file with papers to be identified
papers = pd.read_csv('sample_data/dev_sample.csv')

In [ ]:
# Define the template with placeholders for the columns
query_template = "Title: {Publication Title}"                     # Any Division Match = 0.7, Any Group Match = 0.4
# query_template = "Paper Title: {Publication Title}"                 # Any Division Match = 0.6, Any Group Match = 0.4
# query_template = "{Publication Title}"                            # Any Division Match = 0.6, Any Group Match = 0.4
# query_template = "{OUTLET}, {Publication Title}"                  # Any Division Match = 0.5, Any Group Match = 0.5
# query_template = "Journal: {OUTLET}, Title: {Publication Title}"  # Any Division Match = 0.5, Any Group Match = 0.4
# query_template = "Title: {Publication Title}, Journal: {OUTLET}"  # Any Division Match = 0.5, Any Group Match = 0.4
# query_template = "{OUTLET}, {Publication Title}, {DESCRIPTION}"   # Any Division Match = 0.4, Any Group Match = 0.4
# query_template = "Title: {Publication Title}, Faculty: {Author Department}" # Any Division Match = 0.3, Any Group Match = 0.1
# query_template = "{Author Faculty}, Title: {Publication Title}"   # Any Division Match = 0.4, Any Group Match = 0.3

In [ ]:
# Call the function, passing the template as an argument
output = utils.classify_papers_with_template(
    papers, 
    classifier, 
    hierarchy_with_ids, 
    template=query_template, 
    threshold=0.7, 
    n_max=3
)

In [ ]:
# Define the match configuration with match types and number of digits
match_config = {
    'Division': 2,
    'Group': 4
}

# Apply the function to the DataFrame
matched = utils.apply_code_matches(output, match_config)

In [ ]:
# Write out the csv
matched.to_csv(f'./output/dev_{query_template}.csv', index=False)

In [ ]:
# Calculate performance metrics
# Fraction matched at the division level
print(matched['Any_Division_Match'].mean())

In [ ]:
# Fraction matched at the group level
print(matched['Any_Group_Match'].mean())